In [9]:
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.image as mpimg
import random
from PIL import Image
import collections as co
import cv2
import scipy as sp
import copy

/Users/chrisheinrich/anaconda/envs/deep/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [36]:
import os
import shutil
import errno 

home_dir = os.getcwd()
original_dataset_dir = os.path.join(home_dir, 'kaggle_train')


def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
segmented_dir = os.path.join(home_dir, 'segmented')
mkdir_p(segmented_dir)

segmented_dir_2 = os.path.join(home_dir, 'segmented_2')
mkdir_p(segmented_dir_2)

masked_dir = os.path.join(home_dir, 'masked')
mkdir_p(masked_dir)

In [20]:
image_list = os.listdir(original_dataset_dir)

In [ ]:
segmented_count = 4000

for i in range(segmented_count):
    imgPath = os.path.join(original_dataset_dir, image_list[i])
    img = cv2.imread(imgPath)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

  # subtract background
    height, width, channels = img.shape
    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)

    rect = (0,0,width-1,height-1)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    maskBG = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img = img*maskBG[:,:,np.newaxis]
    segImgPath = os.path.join(segmented_dir_2, image_list[i])
    cv2.imwrite(segImgPath,img)
    
    black_bg = np.zeros((height,width), np.uint8)
    maskTail = np.where((mask==2)|(mask==0),1,0).astype('uint8')
    maskWithBG = black_bg + maskTail
    maskImgPath = os.path.join(masked_dir, image_list[i])
    cv2.imwrite(maskImgPath,maskWithBG)
    

In [34]:
testImgPath = os.path.join(original_dataset_dir,'0ae90da3.jpg')

src = cv2.imread(testImgPath)
print (src.shape)
blank_image = np.zeros(src.shape, np.uint8)
output = cv2.watershed(src,blank_image)

(700, 1050, 3)


error: /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/segmentation.cpp:161: error: (-215) src.type() == (((0) & ((1 << 3) - 1)) + (((3)-1) << 3)) && dst.type() == (((4) & ((1 << 3) - 1)) + (((1)-1) << 3)) in function watershed
